In [23]:
import json
import os
from datetime import datetime, timedelta

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from crewai_tools import CSVSearchTool

from langchain_openai import ChatOpenAI

import yfinance as yf

In [ ]:
os.environ["OPENAI_API_KEY"] = token
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [15]:
csvWalletTool = CSVSearchTool(csv=r"C:\Users\julia\OneDrive\Área de Trabalho\IA para Devs\AI-agent\Wallet\wallet.csv")

In [16]:
customerManager = Agent(
    role="Customer Stock Manager",
    goal="Get the customer question about the stock {ticket} and search the customer wallet CSV for the stocks.",
    backstory="""You are the manager of the customer investments wallet.
    You are the client first contact and you the other analystis with the necessary stock ticket and wallet information.
    """,
    verbose=True, 
    llm=llm,
    max_iter=5,
    tools=[csvWalletTool],
    allow_delegation=False,
    memory=True
)

c:\Users\julia\OneDrive\Área de Trabalho\IA para Devs\AI-agent\venv\Lib\site-packages\langchain_core\tools\structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  return self.args_schema.schema()["properties"]


In [19]:
getCustomerWallet= Task(
    description="""Use the customer question and find the {ticket} in the CSV file.
    Provide if the stock is in the customer wallet and if it is. provide with the mean price he paid
    and the number of stocks he owns.""",
    expected_output="""If the customer owns the stocks, provide the mean price paid and the total stock numbers.""",
    agent=customerManager
)

In [20]:
stocketPriceAnalyst = Agent(
    role= "Senior Stock Price Analyst",
    goal="Find the {ticket} stock price and analyses price trends. Compare with the proce that the customer paid.", 
    backstory="""You're a highly experienced in analysing the price of specific stocks and
    make predictions about its future price.
    """,
    verbose=True, 
    llm=llm,
    max_iter=5,
    tools=[csvWalletTool],
    allow_delegation=False,
    memory=True
)

c:\Users\julia\OneDrive\Área de Trabalho\IA para Devs\AI-agent\venv\Lib\site-packages\langchain_core\tools\structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  return self.args_schema.schema()["properties"]


In [24]:
#yahoo finance tool
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2024-10-07", end="2024-10-08")
    return stock

In [27]:
result = fetch_stock_price("AMZN")

2025-11-16 11:36:35,660 - 3324 - base.py-base:128 - ERROR: Failed to get ticker 'AMZN' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed
2025-11-16 11:36:35,666 - 8060 - multi.py-multi:171 - ERROR: 
1 Failed download:
2025-11-16 11:36:35,667 - 8060 - multi.py-multi:184 - ERROR: ['AMZN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [28]:
result

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
getStockPrice= Task(
    description="Analyze the stock {ticket} price history and create a price trend analyses of up, down or sideways.",
    expected_output="""Specify the current trend stocks price - Up, down or sideways.
    eg. 'stock=AAPL, price UP'.
    """,
    tools=[],
    agent=stocketPriceAnalyst
)